# Next Item

In [1]:
import warnings
warnings.simplefilter('ignore')

import gc
import re
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm

In [2]:
df_sess = pd.read_csv('data/sessions_train.csv')
df_sess

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
2272517,['B00I46IIFQ' 'B0007SQEPK'],B0000E5SEQ,UK
2272518,['B09JWPQ7TC' 'B017OUFLMA'],B076HWCG63,UK
2272519,['B0B81KX8XK' 'B091F65ZYX' 'B07G71F8NX' 'B0816...,B0B4DLXQTW,UK
2272520,['B09W5ZJZ6P' 'B09W5VR5DS' 'B09W5VZ7PQ' 'B09W5...,B09XB864PV,UK


In [3]:
df_test = pd.read_csv('data/sessions_test_task1.csv')
df_test

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE
...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK
316969,['B01MCQMORK' 'B09JYZ325W'],UK


In [4]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [5]:
next_item_dict = defaultdict(list)

for _, row in tqdm(df_sess.iterrows(), total=len(df_sess)):
    prev_items = str2list(row['prev_items'])
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        next_item_dict[prev_items[0]].append(next_item)
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])
        next_item_dict[prev_items[-1]].append(next_item)

  0%|          | 0/2272522 [00:00<?, ?it/s]

In [6]:
for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    prev_items = str2list(row['prev_items'])
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

  0%|          | 0/316971 [00:00<?, ?it/s]

In [7]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

  0%|          | 0/1074040 [00:00<?, ?it/s]

In [8]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B09W9FND7K,B09JSPLN1M
1,B09W9FND7K,B09JSPLN1M
2,B09W9FND7K,B09JSPLN1M
3,B09W9FND7K,B09JSPLN1M
4,B09W9FND7K,B078WW2WN5
...,...,...
11009677,B09SZMLQ9T,B075JHKSGJ
11009678,B081TXFKS5,B004RN5I6W
11009679,1402277822,1402277822
11009680,1402277822,B00APIVOIE


In [9]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]

In [10]:
df_test['last_item'] = df_test['prev_items'].apply(lambda x: str2list(x)[-1])
df_test['next_item_prediction'] = df_test['last_item'].map(next_item_map)
df_test

,prev_items,locale,last_item,next_item_prediction
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE,B099NQFMG7,"[B099NS1XPG, B08496TCCQ, B01BVG1XJS, B099NR3X6..."
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE,B00R9RZ9ZS,"[B004ZXMV4Q, B00R9R5ND6, B095TQTZXY, B086J6RTT..."
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE,B07G7Q5N6G,"[B08C9Q7QVK, B07G7Q5N6G, B07YSRXJD3, B0B5QNFWJ..."
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE,3955350843,"[395535086X, 3955350843, B0829LZFT1, 377247695..."
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE,B09J945WQR,"[B09J8V18FL, B09J8T6TTH, B09J8SKX9G, B09J8TWRV..."
...,...,...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK,B0B14M3VZX,[B08X9L5RGD]
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK,B081YDH55K,"[B09CPNS7XV, B0989BHLSY, B09895QPQF, B09CPP92Q..."
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK,B09HL11V5B,"[B09HKZBNZH, B09HL141QC, B09HZSRJWW, B07Q1ZMNZ..."
316969,['B01MCQMORK' 'B09JYZ325W'],UK,B09JYZ325W,"[B08JG8TSCP, B07TR5LQSL, B09JYZ325W]"


# Fill up with FAISS

In [11]:
# next items preds
df_test_de = df_test[df_test["locale"] == "DE"]
df_test_uk = df_test[df_test["locale"] == "UK"]
df_test_jp = df_test[df_test["locale"] == "JP"]

In [12]:
# faiss preds
preds_de = pd.read_csv("pred_sessions_test_task1_de.csv", index_col=[0])
preds_uk = pd.read_csv("pred_sessions_test_task1_jp.csv", index_col=[0])
preds_jp = pd.read_csv("pred_sessions_test_task1_uk.csv", index_col=[0])

display(preds_de.head())
display(preds_uk.head())
display(preds_jp.head())

,prev_items,locale,next_item
0,"['B08V12CT4C', 'B08V1KXBQD', 'B01BVG1XJS', 'B0...",DE,"['B08V1KXBQD', 'B07TV22X9M', 'B08V12CT4C', 'B0..."
1,"['B00R9R5ND6', 'B00R9RZ9ZS', 'B00R9RZ9ZS']",DE,"['B00R9RZ9ZS', 'B00R9R5ND6', 'B095TL42FL', '37..."
2,"['B07YSRXJD3', 'B07G7Q5N6G', 'B08C9Q7QVK', 'B0...",DE,"['B07G7Q5N6G', 'B08C9Q7QVK', 'B099277D7Q', 'B0..."
3,"['B08KQBYV43', '3955350843', '3955350843', '39...",DE,"['3741524131', '3955350878', '395535086X', 'B0..."
4,"['B09FPTCWMC', 'B09FPTQP68', 'B08HMRY8NG', 'B0...",DE,"['B09J8V18FL', 'B09J8VPTTW', 'B09J8TTZ68', 'B0..."


,prev_items,locale,next_item
104568,"['B09NZHMKXW', 'B0823M5W31', 'B017GSMOJS', 'B0...",JP,"['B08NZN9WC1', 'B09S25SZ1Q', 'B06Y4F4NDS', 'B0..."
104569,"['B08S31PW4G', 'B09TKN5SGL']",JP,"['B09BHP5PP1', 'B07F3WG12T', 'B09CG7D7LN', 'B0..."
104570,"['B0BHWBZ4ZJ', 'B0BHWBZ4ZJ', 'B09MQCDRZ1', 'B0...",JP,"['B09V7RBRWN', 'B000J3VKLQ', 'B09CD5BQ7Y', 'B0..."
104571,"['B07VHJZQYB', 'B07Y1P1B5T']",JP,"['B087CQM9BT', 'B089SWFFFV', 'B07VCLCJT3', 'B0..."
104572,"['B09JK112HQ', 'B07GCTJ7JX', 'B0BKK2NRPJ', 'B0...",JP,"['B09FSNXWKP', 'B0926NWR4L', 'B09QXDVQ1V', 'B0..."


,prev_items,locale,next_item
201035,"['B007MO0FIO', 'B09QXPN2TL', 'B005PA3I8G', 'B0...",UK,"['B091TLGK2T', 'B0BFX4PPYM', 'B08WPCZ8KR', 'B0..."
201036,"['B082P6M3HC', 'B082BHJ9G8', 'B0797QCXS6', 'B0...",UK,"['B0791BS4NL', 'B09SH64F6S', 'B07S7CZZ7L', 'B0..."
201037,"['B0894HWG1H', 'B07S79G11S', 'B092VCHVS7', 'B0...",UK,"['B0B7RW8HYV', 'B07TS1NXSH', 'B07MMWKWH1', 'B0..."
201038,"['B00FLTE8OE', 'B07CRNCKMR', 'B0784T65RK']",UK,"['B0B717ZVD5', 'B006L4DLW8', 'B091F4QN7K', 'B0..."
201039,"['B0B5XPGRXD', 'B0BHSXQLSX', 'B0BJKJZJNM']",UK,"['3742322508', 'B095XWVYMN', 'B001QCELXS', 'B0..."


In [13]:
df_test_de['next_item_prediction'] = df_test_de['next_item_prediction'].replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
df_test_uk['next_item_prediction'] = df_test_uk['next_item_prediction'].replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
df_test_jp['next_item_prediction'] = df_test_jp['next_item_prediction'].replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')

In [14]:
preds_de = list(preds_de['next_item'].str.strip("[']").str.split("', '"))
preds_uk = list(preds_uk['next_item'].str.strip("[']").str.split("', '"))
preds_jp = list(preds_jp['next_item'].str.strip("[']").str.split("', '"))

In [15]:
# preds_combined_de = df_test_de + preds_de
# preds_combined_uk = df_test_uk + preds_uk
# preds_combined_jp = df_test_jp + preds_jp

ValueError: Unable to coerce to Series, length must be 4: given 104568

In [16]:
def drop_duplicates_from_list(original_list):
    unique_list = []
    for element in original_list:
        if element not in unique_list:
            unique_list.append(element)
    return unique_list

In [19]:
# fill up next_item lists that are shorter than 100 items with most common items
def fill_up(next_items, locale):
    n_items_missing = 100 - len(next_items)
    
    if n_items_missing > 0:
        if locale == "DE":
            next_items = next_items + preds_de
            next_items = drop_duplicates_from_list(next_items)
            if len(next_items) < 100:
                print("DE missing")
            return next_items[:100]
        if locale == "UK":
            next_items = next_items + preds_uk
            next_items = drop_duplicates_from_list(next_items)
            if len(next_items) < 100:
                print("UK missing")
            return next_items[:100]
        if locale == "JP":
            next_items = next_items + preds_jp
            next_items = drop_duplicates_from_list(next_items)
            if len(next_items) < 100:
                print("JP missing")
            return next_items[:100]
    else:
        return next_items

In [ ]:
df_test_de["next_item_prediction"] = df_test_de["next_item_prediction"].apply(lambda l: fill_up(l, "DE"))
df_test_uk["next_item_prediction"] = df_test_uk["next_item_prediction"].apply(lambda l: fill_up(l, "UK"))
df_test_jp["next_item_prediction"] = df_test_jp["next_item_prediction"].apply(lambda l: fill_up(l, "JP"))